# домашнее задание №2
## Анастасия Сафонова

# Задание 1

Реализуйте триграмную модель и сгенерируйте несколько текстов. Сравните их с текстами, сгенерированными биграмной моделью. 
Можно использовать те же тексты, что в семинаре, или взять какой-то другой (на английском или русском языке).  

In [82]:
news = open('data/lenta.txt').read()[:300000]

In [83]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


In [84]:
norm_news = normalize(news)

In [85]:
from collections import Counter

In [86]:
vocab_news = Counter(norm_news)

In [87]:
from nltk.tokenize import sent_tokenize
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

#### Сначала сгенерируем биграммы

In [88]:
sentences_news_bi = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)]

In [90]:
unigrams_news = Counter()
bigrams_news = Counter()

for sentence in sentences_news_bi:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))

In [93]:
matrix_news_bi = np.zeros((len(unigrams_news), 
                   len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}


# вероятность расчитываем точно также
for ngram in bigrams_news:
    word1, word2 = ngram.split()
    matrix_news_bi[word2id_news[word1]][word2id_news[word2]] =  (bigrams_news[ngram]/
                                                                     unigrams_news[word1])

In [94]:
def generate_bi(matrix, id2word, word2id, n=100, start='<start>'):
    text = []
    current_idx = word2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = word2id['<start>']
        current_idx = chosen
    
    return ' '.join(text)

In [95]:
print(generate_bi(matrix_news_bi, id2word_news, word2id_news).replace('<end>', '\n'))

таких автомобилей ford терри бресниан сообщил корреспонденту итар-тасс в печатниках московский гарнизон милиции 
 в силу советских танков введенных в случае победила 
 полтора года указом президента фирмы mabetex были однако не принял экстренные меры не исключил фактор мщения со ссылкой на земле колыхаясь знамена шумели и с премьер-министром японии еще больше у мвф 
 земля на 10 лет представители американских банках 
 национальная авиакомпания олимпик предложила бесплатные билеты для нормальной работы 
 когда тот думает о статусе иерусалима о себе в дагестане министр внутренних и методами 
 велению рока послушный противник 
 сразу на ленинском проспекте юго-западный администивный округ


#### Теперь сгенерируем триграммы

In [96]:
sentences_news_tri = [['<start>', '<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)]

In [98]:
unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news_tri:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))
    trigrams_news.update(ngrammer(sentence, n=3))

In [100]:
matrix_news_tri = np.zeros((len(bigrams_news), len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}

id2bigram_news = list(bigrams_news)
bigram2id_news = {word:i for i, word in enumerate(id2bigram_news)}


for ngram in trigrams_news:
    first_word, second_word, third_word = ngram.split()
    key = ' '.join([first_word, second_word])
    matrix_news_tri[bigram2id_news[key]][word2id_news[third_word]] = (trigrams_news[ngram]/bigrams_news[key])

In [101]:
def generate_tri(matrix, id2word, word2id, n=100, start='<start> <start>'):
    text = []
    current_idx = word2id[start]
    first_word, second_word = start.split()
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            current_idx = word2id[start]
            first_word, second_word = start.split()
        else:
            next_word = ' '.join([second_word, id2word[chosen]])
            current_idx = word2id[next_word]
            second_word = id2word[chosen]
    
    return ' '.join(text)

In [102]:
print(generate_tri(matrix_news_tri, id2word_news, bigram2id_news).replace('<end>', '\n'))

коржаков напомнил что в настоящее время в главной миссии оон передаваемым риа новости руководитель пресс-службы мвд по москве и московской области 
 9 сентября не будет больше совершать теракты против турции сообщает bbc президент азербайджана гейдар алиев поручил разработатку и создание газопровода для доставки природного газа за рубеж по каналам bank of new york якобы связанные с борисом ельциным 
 в ликвидации последствий взрыва пострадали получив различные травмы 
 этот архив был доставлен в военный госпиталь на территории россии и чечни он сообщил также что не существует каких-либо плановограничить степень информированности людей о событиях на северном кавказе обвиняя во всем


### Текст, сгенирированный по триграммам, выглядит чуть более осмысленным, чем тот, что создан по биграммам

# Задание 2

Напишите функцию оценивания нграммов на основе PMI. Используйте эту функцию вместо дефолтной в gensim.models.Phrases Обучите два последовательных модели Phrases с такой функцией и проанализируйте результаты, получаемые после преобразования текстов двумя Phrases.

In [57]:
import itertools
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
from pymorphy2 import MorphAnalyzer
from collections import Counter, defaultdict
import numpy as np
import re
from string import punctuation
import gensim
from nltk.corpus import stopwords

stops = set(stopwords.words('russian') + ["это", "весь"])
morph = MorphAnalyzer()

def normalize(text):
    tokens = re.findall('[а-яёa-z0-9]+', text.lower())
    normalized_text = [morph.parse(word)[0].normal_form for word \
                                                            in tokens]
    normalized_text = [word for word in normalized_text if len(word) > 2 and word not in stops]
    
    return normalized_text

def preprocess(text):
    sents = sentenize(text)
    return [normalize(sent.text) for sent in sents]

def ngrammer(tokens, stops, n=2):
    ngrams = []
    tokens = [token for token in tokens if token not in stops]
    for i in range(0,len(tokens)-n+1):
        ngrams.append(tuple(tokens[i:i+n]))
    return ngrams

In [59]:
corpus = open('data/lenta.txt').read()[:500000]
corpus = preprocess(corpus)

In [60]:
def scorer(worda_count, wordb_count, bigram_count, len_vocab, min_count, corpus_word_count):
    try:
        score = np.log((bigram_count / corpus_word_count) / ((worda_count / corpus_word_count) * (wordb_count / corpus_word_count)))
    except ZeroDivisionError:
        return 0
    
    return score

In [76]:
ph = gensim.models.Phrases(corpus, scoring=scorer, threshold=0)
p = gensim.models.phrases.Phraser(ph)
ph2 = gensim.models.Phrases(p[corpus], scoring=scorer, threshold=0)
p2 = gensim.models.phrases.Phraser(ph2)
p2[p[corpus[333]]]

['установить_взрыв_произойти_третий',
 'уровнечетвертый_ярус_комплекс_зал',
 'игровой_автомат']

In [77]:
ph = gensim.models.Phrases(corpus, scoring=scorer, threshold=3)
p = gensim.models.phrases.Phraser(ph)
ph2 = gensim.models.Phrases(p[corpus], scoring=scorer, threshold=3)
p2 = gensim.models.phrases.Phraser(ph2)
p2[p[corpus[333]]]

['установить_взрыв_произойти_третий',
 'уровнечетвертый_ярус_комплекс_зал',
 'игровой_автомат']

In [78]:
ph = gensim.models.Phrases(corpus, scoring=scorer, threshold=5)
p = gensim.models.phrases.Phraser(ph)
ph2 = gensim.models.Phrases(p[corpus], scoring=scorer, threshold=5)
p2 = gensim.models.phrases.Phraser(ph2)
p2[p[corpus[333]]]

['установить',
 'взрыв',
 'произойти_третий_уровнечетвертый',
 'ярус_комплекс_зал_игровой',
 'автомат']

In [79]:
ph = gensim.models.Phrases(corpus, scoring=scorer, threshold=8)
p = gensim.models.phrases.Phraser(ph)
ph2 = gensim.models.Phrases(p[corpus], scoring=scorer, threshold=8)
p2 = gensim.models.phrases.Phraser(ph2)
p2[p[corpus[333]]]

['установить',
 'взрыв',
 'произойти',
 'третий_уровнечетвертый_ярус',
 'комплекс',
 'зал_игровой_автомат']

In [80]:
ph = gensim.models.Phrases(corpus, scoring=scorer, threshold=10)
p = gensim.models.phrases.Phraser(ph)
ph2 = gensim.models.Phrases(p[corpus], scoring=scorer, threshold=10)
p2 = gensim.models.phrases.Phraser(ph2)
p2[p[corpus[333]]]

['установить',
 'взрыв',
 'произойти',
 'третий',
 'уровнечетвертый_ярус',
 'комплекс',
 'зал',
 'игровой',
 'автомат']

In [81]:
ph = gensim.models.Phrases(corpus, scoring=scorer, threshold=13)
p = gensim.models.phrases.Phraser(ph)
ph2 = gensim.models.Phrases(p[corpus], scoring=scorer, threshold=13)
p2 = gensim.models.phrases.Phraser(ph2)
p2[p[corpus[333]]]

['установить',
 'взрыв',
 'произойти',
 'третий',
 'уровнечетвертый',
 'ярус',
 'комплекс',
 'зал',
 'игровой',
 'автомат']

### я не уверена, что сделала это задание правильно, потому что результат получается очень странный. Вероятно, лучший threshold должен быть 8 или 10, тогда получается хоть что-то отдаленно напоминающее коллокации. Если он больше, то всё распадается на одиночные слова, если меньше -- будто бы рандомно соединяютюся